<a href="https://colab.research.google.com/github/imharshmishra87/Feature-Scaling-Ml/blob/main/Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import pandas as pd

In [9]:
df=pd.read_csv("/content/Titanic-Dataset.csv")
df.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.100,C123,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.000,B51 B53 B55,S
392,393,0,3,"Gustafsson, Mr. Johan Birger",male,28.0,2,0,3101277,7.925,NaN,S
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S
196,197,0,3,"Mernagh, Mr. Robert",male,NaN,0,0,368703,7.750,NaN,Q


In [10]:
df=df[['Name','Sex','Age','Parch','Embarked','Pclass','Survived']]
df.head()
df.shape

(891, 7)

# Train, Test and Split Of Data

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,1:6],df.iloc[:,-1:],random_state=True,test_size=0.1)


# Handling Missing Values

In [12]:
from sklearn.impute import SimpleImputer
si=SimpleImputer(strategy='mean')
X_train[['Age']]=si.fit_transform(X_train[['Age']])
X_test[['Age']]=si.fit_transform(X_test[['Age']])

# One Hot Encoding

In [13]:
from sklearn.preprocessing import OneHotEncoder
oe=OneHotEncoder(sparse_output=False,drop='first')
oe.fit(X_train[['Sex','Embarked']])
X_train_new=oe.transform(X_train[['Sex','Embarked']])
X_test_new=oe.transform(X_test[['Sex','Embarked']])

# Converting numpy array Into Pandas Data Frame

In [15]:
# Horizontally Stacking The Numpy arrays
X_train_scaled=np.hstack((X_train[['Age','Parch','Pclass']].values,X_train_new))
X_test_scaled=np.hstack((X_test[['Age','Parch','Pclass']].values,X_test_new))

# Extracting The Coloumns Names And Adding Names Back
new_column=oe.get_feature_names_out(['Sex','Embarked'])
combined_column=['Age','Parch','Pclass']+ list(new_column)

# Converting Data Into Data Frame
X_train_scaled=pd.DataFrame(X_train_scaled, columns=combined_column)
X_test_scaled=pd.DataFrame(X_test_scaled, columns=combined_column)
X_train_scaled

,Age,Parch,Pclass,Sex_male,Embarked_Q,Embarked_S,Embarked_nan
0,47.000000,0.0,1.0,1.0,0.0,1.0,0.0
1,29.859641,0.0,3.0,1.0,0.0,0.0,0.0
2,25.000000,0.0,3.0,1.0,1.0,0.0,0.0
3,29.859641,0.0,3.0,0.0,1.0,0.0,0.0
4,29.859641,0.0,3.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...
796,19.000000,0.0,3.0,1.0,0.0,1.0,0.0
797,30.500000,0.0,3.0,0.0,1.0,0.0,0.0
798,21.000000,0.0,2.0,1.0,0.0,1.0,0.0
799,29.859641,0.0,3.0,0.0,0.0,1.0,0.0


# Using Column Transformer

In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
ct=ColumnTransformer(transformers=[('ts1',SimpleImputer(),['Age']),('ts2',OneHotEncoder(sparse_output=False, drop='first'),['Sex','Embarked'])],remainder='passthrough')
X_train_encoded=ct.fit_transform(X_train)
X_test_encoded=ct.transform(X_test)

# Training And Prediction Of Model

In [19]:
from sklearn.linear_model import LogisticRegression
le=LogisticRegression()
le.fit(X_train_encoded,y_train)
y_pred=le.predict(X_test_encoded)

from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test.values.ravel(), y_pred)*100
print("Model has an accuracy of {} %".format(accuracy))


Model has an accuracy of 75.55555555555556 %


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
